## <center> LGBM Cross Validation

In [3]:
%store -r __ipy
%store -r __da
%store -r __ml

In [4]:
__ipy

Helper ipython script loaded


In [5]:
__da

Basic Data Analysis tools was loaded


In [6]:
__ml

In [ ]:
def generate_and_merge(X_train, X_test, settings):
    features = generate_ts_features(X_train, 'meter_reading_log1p', 'hour_datetime', settings)
    
    X_train = X_train.merge(features, how='left', left_on=KEY_NAME, right_on='id')
    X_test = X_test.merge(features, how='left', left_on=KEY_NAME, right_on='id')
    
    return X_train, X_test


settings  = load_settings('')
train_X, test_X, tsfresh_features = read_features(generate_and_merge, settings, test=False)

In [6]:
folds = 6
@telegram_sender(token=TELEGRAM_API_KEY, chat_id=CHAT_ID)
def lgbm_cross_validation(params):
    seed = 42
    
    kf = KFold(n_splits=folds)
    total_loss = []
    
    for train_index, val_index in kf.split(full_train_df):
        
        train_X = full_train_df.loc[train_index, [*feat_cols, 'meter_reading_log1p']].reset_index(drop=True)
        val_X = full_train_df.loc[val_index, feat_cols].reset_index(drop=True)
        train_y = target.iloc[train_index]
        val_y = target.iloc[val_index]
        
        ###
        
        train_X, val_X = generate_and_merge(train_X, val_X, settings)
        
        ###
        train_X.drop('meter_reading_log1p', axis=1, inplace=True)
        ####
        
        ####
        
        lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
        lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)
        lgbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    valid_sets=(lgb_train, lgb_eval),
                    early_stopping_rounds=100,
                    verbose_eval = 0)
        
        pred_y = lgbm.predict(val_X)
        mse = np.sqrt(mean_squared_error((val_y), (pred_y)))
        total_loss.append(mse)

    return {'loss': np.mean(total_loss), 'status': STATUS_OK, 'params': params }

def optimize_lgbm(max_evals=1000):
    space = {
        'metric': {'rmse'},
        'num_leaves': scope.int(hp.quniform('num_leaves', 30, 150, 1)),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
        'min_data_in_leaf': scope.int(hp.qloguniform('min_data_in_leaf', 0, 6, 1)),
        'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
        'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)]),
        'min_child_weight': hp.loguniform('min_child_weight', -16, 5), #also aliases to min_sum_hessian
        'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
        'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    }
    
    trials = Trials()
    best = fmin(fn=lgbm_cross_validation,
                space=space,
                algo=tpe.suggest,
                max_evals=max_evals,
                trials=trials,
                verbose= 1)

    # find the trial with lowest loss value. this is what we consider the best one
    idx = np.argmin(trials.losses())
    print(idx)

    print(trials.trials[idx])
    # these should be the training parameters to use to achieve the best score in best trial
    params = trials.trials[idx]["result"]["params"]
    
    print(params)
    return params